In [1]:
from pymongo import MongoClient
import time
import operator
import datetime
import re

client = MongoClient()
db = client.twitter
te = db['prueba2']
dia_inicio = datetime.datetime.now()
dia_inicio = datetime.datetime(dia_inicio.year, dia_inicio.month, 25)
#dia_inicio = te.find_one(sort=[("created_at", 1)])["created_at"]
print dia_inicio

#fin = te.find_one(sort=[("created_at", -1)])["created_at"]
fin = datetime.datetime(dia_inicio.year, dia_inicio.month, 26)
print fin

2017-04-25 00:00:00
2017-04-26 00:00:00


In [2]:
def insert_web_dict(d,value):
    if not value:
        return
    if 'https://twitter.com/' in value:
        return
    if value in d:
        d[value] = d[value] + 1
    else:
        d[value] = 1


        
    

In [3]:
def get_trending(init, end, limit):
    dw = {}
    count = 0
    for a in te.find({'$and': [{'created_at': {"$gte" : init}} ,{'created_at': {"$lt" : end}} ]}):
        count = count +1
        if len(a['entities']['urls']) >0:
            for b in a['entities']['urls']:
                insert_web_dict(dw,b['expanded_url'])
        if 'retweeted_status' in a and len(a['retweeted_status']['entities']['urls']) >0:      
            for b in a['retweeted_status']['entities']['urls']:
                insert_web_dict(dw,b['expanded_url'])
    trending = sorted(dw.items(), key=operator.itemgetter(1), reverse=True)
    return trending[:limit], count

In [4]:
intervalo = 60
a =  datetime.datetime(dia_inicio.year, dia_inicio.month, dia_inicio.day, dia_inicio.hour, dia_inicio.minute)
while fin > a:
    b = a + datetime.timedelta(minutes=intervalo)
    (tr, count)= get_trending(a, b, 5)
    if count < 3000:
        tr = []
    first = True
    for w in tr:
        if w:
            if first:
                print str(a) + ':'
                first = False
            print '\t' + str(w)
    a = b

In [49]:
###Creamos rangos y cogemos lo más visitado
d = {}
for a in te.find({'$and': [{'text':{"$regex" : '#'}} ,{'created_at': {"$gt" : dia_inicio}} ]},  { 'text': 1}):
	s = a['text']
	for ht in  [b.strip('#') for b in s.split() if b.startswith("#")]:
		if ht not in d:
			d[ht] = 1
		else:
			d[ht] = d[ht] + 1
trending = sorted(d.items(), key=operator.itemgetter(1), reverse=True)
for a in trending:
    print a

In [83]:
from collections import defaultdict
limit = 100
dw2 = defaultdict(lambda: 0)
for a in te.find():
    lh = []
    for b in a['entities']['hashtags']:
        lh.append(b['text'])
    if 'retweeted_status' in a:   
        for b in a['retweeted_status']['entities']['hashtags']:
            lh.append(b['text'])
    
    if len(a['entities']['urls']) >0:
        for b in a['entities']['urls']:
            if dw2[b['expanded_url']]:
                dw2[b['expanded_url']] += 1
    if 'retweeted_status' in a and len(a['retweeted_status']['entities']['urls']) >0:      
        for b in a['retweeted_status']['entities']['urls']:
                dw2[b['expanded_url']] +=  1
            
    
#    if len(a['entities']['urls']) >0:
#        for b in a['entities']['urls']:
#            if b['expanded_url'] in dw2:
#                dw2[b['expanded_url']] = dw2[b['expanded_url']] + lh
#            else:
#                dw2[b['expanded_url']] = lh
#    if 'retweeted_status' in a and len(a['retweeted_status']['entities']['urls']) >0:      
#        for b in a['retweeted_status']['entities']['urls']:
#            if b['expanded_url'] in dw2:
#                dw2[b['expanded_url']] = dw2[b['expanded_url']] + lh
#            else:
#                dw2[b['expanded_url']] = lh
            
trending = sorted(dw2.items(), key=operator.itemgetter(1), reverse=True)
i = 1
urls = [] 
medias = ['elpais', 'elmundo', 'eleconomista', 'elmundotoday', 'marca com', '20minutos', 'meneame', 'quora']
for a in trending:
    if any(word in a[0] for word in medias):
        urls.append(a)
        i = i+1
    if i > limit:
        break

In [84]:
from urllib import urlopen
from bs4 import BeautifulSoup

def get_data_pais(name):
    webpage = urlopen(name).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    title = soup.find("meta",  property="og:title")
    url = soup.find("meta",  property="og:url")
    desc = soup.find("meta",  property="og:description")
    img = soup.find("meta",  property="og:image")
    d = {}
    d['title'] = title["content"] if title else None
    d['url'] = url["content"] if url else None
    d["desc"] = desc["content"] if desc else None
    d["img"] = img["content"] if img else None
    #print(title["content"] if title else "No meta title given")
    #print(url["content"] if url else "No meta url given")
    #print(desc["content"] if desc else "No meta desc given")
    #print(img["content"] if img else "No meta image given")
    return d
  
lurls = []
for url in urls:
    d = get_data_pais(url[0])
    d['rank'] = url[1]
    lurls.append(d)
    #print '------------------'

    
  


In [85]:
with open('tweevy', 'w') as f:
    f.write(json.dumps(lurls))